In [2]:
import numpy as np
!pip install geocoder
import pandas as pd
!pip install folium
import json
from geopy.geocoders import Nominatim
import geocoder
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import folium
import seaborn as sns
%matplotlib inline
print("Libraries imported.")

     |████████████████████████████████| 98 kB 8.2 MB/s  eta 0:00:01
     |████████████████████████████████| 93 kB 3.5 MB/s  eta 0:00:01
Libraries imported.


In [3]:
dt=requests.get("https://en.wikipedia.org/wiki/Boroughs_and_neighborhoods_of_Berlin")

In [4]:
soup=BeautifulSoup(dt.content, 'html.parser')

In [5]:
table=soup.find_all('table', class_='nowraplinks')

In [6]:
ul=table[0].find_all('ul')

In [7]:
df = pd.DataFrame (columns = ['Borough','Neighborhood','Latitude','Longitude'])

In [8]:
c=0
for i in range(1,len(table[0].find_all('th'))):
    for j in range(len(ul[i].find_all('li'))):
        df.loc[c,'Borough']=table[0].find_all('th')[i].get_text()
        df.loc[c,'Neighborhood']=ul[i].find_all('li')[j].get_text()
        c=c+1
df

,Borough,Neighborhood,Latitude,Longitude
0,Charlottenburg-Wilmersdorf,Charlottenburg,NaN,NaN
1,Charlottenburg-Wilmersdorf,Charlottenburg-Nord,NaN,NaN
2,Charlottenburg-Wilmersdorf,Grunewald,NaN,NaN
3,Charlottenburg-Wilmersdorf,Halensee,NaN,NaN
4,Charlottenburg-Wilmersdorf,Schmargendorf,NaN,NaN
...,...,...,...,...
91,Treptow-Köpenick,Niederschöneweide,NaN,NaN
92,Treptow-Köpenick,Oberschöneweide,NaN,NaN
93,Treptow-Köpenick,Plänterwald,NaN,NaN
94,Treptow-Köpenick,Rahnsdorf,NaN,NaN


In [9]:
# Defining a function to get coordinates
def get_latlng(neighborhood):
 
    lat_lng_coords = None

    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Berlin, Germany'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

coords = [ get_latlng(neighborhood) for neighborhood in df["Neighborhood"].tolist()]

In [10]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']
print(df.shape)
df

(96, 4)


,Borough,Neighborhood,Latitude,Longitude
0,Charlottenburg-Wilmersdorf,Charlottenburg,52.51915,13.30639
1,Charlottenburg-Wilmersdorf,Charlottenburg-Nord,52.53048,13.29371
2,Charlottenburg-Wilmersdorf,Grunewald,52.49203,13.28495
3,Charlottenburg-Wilmersdorf,Halensee,52.49005,13.29602
4,Charlottenburg-Wilmersdorf,Schmargendorf,52.47732,13.28798
...,...,...,...,...
91,Treptow-Köpenick,Niederschöneweide,52.45549,13.51328
92,Treptow-Köpenick,Oberschöneweide,52.46237,13.51438
93,Treptow-Köpenick,Plänterwald,52.48439,13.47957
94,Treptow-Köpenick,Rahnsdorf,52.44093,13.68891


In [11]:
address = 'Berlin, Germany'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Berlin, Germany are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Berlin, Germany are 52.5170365, 13.3888599.


In [12]:
# create map of Berlin using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [13]:
CLIENT_ID = '2V0OPVSGJSVCPLNEIBQTXCQVNG2WTKHGG2WHZCNF3USCJOBF' # your Foursquare ID
CLIENT_SECRET = 'UJ5TN032NOVQ02NWRNGOQF1YPF0E3J0BSMS4UPXVNS5B21WK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2V0OPVSGJSVCPLNEIBQTXCQVNG2WTKHGG2WHZCNF3USCJOBF
CLIENT_SECRET:UJ5TN032NOVQ02NWRNGOQF1YPF0E3J0BSMS4UPXVNS5B21WK


In [14]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
df_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Charlottenburg
Charlottenburg-Nord
Grunewald
Halensee
Schmargendorf
Westend
Wilmersdorf
Friedrichshain
Kreuzberg
Alt-Hohenschönhausen
Falkenberg
Fennpfuhl
Friedrichsfelde
Karlshorst
Lichtenberg
Malchow
Neu-Hohenschönhausen
Rummelsburg
Wartenberg
Biesdorf
Hellersdorf
Kaulsdorf
Mahlsdorf
Marzahn
Gesundbrunnen
Hansaviertel
Mitte
Moabit
Tiergarten
Wedding
Britz
Buckow
Gropiusstadt
Neukölln
Rudow
Blankenburg
Blankenfelde
Buch
Französisch Buchholz
Heinersdorf
Karow
Niederschönhausen
Pankow
Prenzlauer Berg
Rosenthal
Stadtrandsiedlung Malchow
Weißensee
Wilhelmsruh
Borsigwalde
Frohnau
Heiligensee
Hermsdorf
Konradshöhe
Lübars
Märkisches Viertel
Reinickendorf
Tegel
Waidmannslust
Wittenau
Falkenhagener Feld
Gatow
Hakenfelde
Haselhorst
Kladow
Siemensstadt
Spandau
Staaken
Wilhelmstadt
Dahlem
Lankwitz
Lichterfelde
Nikolassee
Steglitz
Wannsee
Zehlendorf
Friedenau
Lichtenrade
Mariendorf
Marienfelde
Schöneberg
Tempelhof
Adlershof
Alt-Treptow
Altglienicke
Baumschulenweg
Bohnsdorf
Friedrichshagen
 Grünau


In [17]:
df_venues.shape

(1623, 7)

In [18]:
df_venues.groupby(["Venue Category"]).count().sort_values('Venue',ascending=False).head(20)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
Café,105,105,105,105,105,105
Supermarket,99,99,99,99,99,99
Bakery,64,64,64,64,64,64
Italian Restaurant,63,63,63,63,63,63
Drugstore,54,54,54,54,54,54
Bus Stop,47,47,47,47,47,47
Bar,36,36,36,36,36,36
Ice Cream Shop,35,35,35,35,35,35
German Restaurant,33,33,33,33,33,33


In [19]:
print('There are {} unique categories.'.format(len(df_venues['Venue Category'].unique())))

There are 262 unique categories.


In [20]:
# One hot encoding
df_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")
# Adding neighborhood column back to dataframe
df_onehot['Neighborhoods'] = df_venues['Neighborhood']
# Moving neighbourhood column to the first column
fixed_columns = [df_onehot.columns[-1]] + list(df_onehot.columns[:-1])
df_onehot = df_onehot[fixed_columns]
print(df_onehot.shape)

(1623, 263)


In [21]:
df_onehot

,Neighborhoods,ATM,Accessories Store,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Yemeni Restaurant,Yoga Studio
0,Charlottenburg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Charlottenburg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Charlottenburg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Charlottenburg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Charlottenburg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1618,Schmöckwitz,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1619,Schmöckwitz,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1620,Schmöckwitz,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1621,Schmöckwitz,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
df_grouped = df_onehot.groupby('Neighborhoods').sum().reset_index()
df_grouped

,Neighborhoods,ATM,Accessories Store,African Restaurant,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Yemeni Restaurant,Yoga Studio
0,Grünau,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Adlershof,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Alt-Hohenschönhausen,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,Alt-Treptow,0,0,0,0,0,0,0,0,1,...,0,1,0,1,0,0,0,0,0,0
4,Altglienicke,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,Wilhelmsruh,0,0,0,0,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
92,Wilhelmstadt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
93,Wilmersdorf,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
94,Wittenau,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
len((df_grouped[df_grouped["Italian Restaurant"] > 0]))

41

In [24]:
# Creating a dataframe for Italian Restaurant data only
df_italian = df_grouped[["Neighborhoods","Italian Restaurant"]]
df_italian.sort_values('Italian Restaurant')

,Neighborhoods,Italian Restaurant
0,Grünau,0
36,Hellersdorf,0
37,Hermsdorf,0
38,Johannisthal,0
40,Karow,0
...,...,...
61,Neukölln,3
44,Kreuzberg,3
24,Friedrichshain,3
95,Zehlendorf,3


In [25]:
# Setting the optimum number of clusters
n_clusters=[2,3,4,5]
for i in n_clusters:
    kclusters = i
    df_clustering = df_italian.drop(["Neighborhoods"], 1)
    # Run k-means clustering algorithm
    kmeans = KMeans(n_clusters=kclusters,random_state=0).fit(df_clustering)
    label=kmeans.predict(df_clustering)
    # Calculating the silhoutte score for all clusters
    print('The silhouette score for {} clusters is {}.'.format(i,silhouette_score(df_clustering, label)))

The silhouette score for 2 clusters is 0.7623111393242606.
The silhouette score for 3 clusters is 0.888850456592392.
The silhouette score for 4 clusters is 0.9384469696969697.
The silhouette score for 5 clusters is 0.9895833333333334.


In [26]:
kclusters = 5
df_clustering = df_italian.drop(["Neighborhoods"], 1)
# Run k-means clustering algorithm
kmeans = KMeans(n_clusters=kclusters,random_state=0).fit(df_clustering)

In [27]:
# Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
df_merged = df_italian.copy()
# Add the clustering labels
df_merged["Cluster Labels"] = kmeans.labels_
df_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
df_merged

,Neighborhood,Italian Restaurant,Cluster Labels
0,Grünau,0,0
1,Adlershof,1,2
2,Alt-Hohenschönhausen,0,0
3,Alt-Treptow,2,4
4,Altglienicke,0,0
...,...,...,...
91,Wilhelmsruh,0,0
92,Wilhelmstadt,0,0
93,Wilmersdorf,1,2
94,Wittenau,0,0


In [28]:
 # Adding latitude and longitude values to the existing dataframe
df_merged['Latitude'] = df['Latitude']
df_merged['Longitude'] = df['Longitude']
# Sorting the results by Cluster Labels
df_merged.sort_values(["Cluster Labels"], inplace=True)
pd.set_option('display.max_rows', None)
df_merged

,Neighborhood,Italian Restaurant,Cluster Labels,Latitude,Longitude
0,Grünau,0,0,52.51915,13.30639
70,Reinickendorf,0,0,52.43293,13.31894
30,Hakenfelde,0,0,52.44873,13.43621
69,Rahnsdorf,0,0,52.43895,13.35340
32,Hansaviertel,0,0,52.42472,13.47370
33,Haselhorst,0,0,52.48077,13.43541
34,Heiligensee,0,0,52.41917,13.49370
35,Heinersdorf,0,0,52.59332,13.45805
36,Hellersdorf,0,0,52.61855,13.39017
37,Hermsdorf,0,0,52.63424,13.49753


In [44]:
df_merged.shape

(96, 5)

In [29]:
# Creating the map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# Setting color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['Neighborhood'], df_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat,lon],radius=5,popup=label,color=rainbow[cluster-1],fill=True,fill_color=rainbow[cluster-1],fill_opacity=0.7).add_to(map_clusters)
map_clusters

In [30]:
df_summary=df_merged.drop_duplicates(subset = ["Italian Restaurant"]).sort_values(['Italian Restaurant'],ascending=True)
df_summary

,Neighborhood,Italian Restaurant,Cluster Labels,Latitude,Longitude
0,Grünau,0,0,52.51915,13.30639
25,Frohnau,1,2,52.51679,13.33835
42,Kladow,2,4,52.56925,13.40248
68,Prenzlauer Berg,3,1,52.45874,13.28344
72,Rudow,5,3,52.45784,13.31808


In [31]:
print('There are {} neighborhoods with 0 Italian Restaurants, they belong to cluster {}'.format(len(df_merged.loc[df_merged['Cluster Labels'] == 0]),0))
print('There are {} neighborhoods with 1 Italian Restaurants, they belong to cluster {}'.format(len(df_merged.loc[df_merged['Cluster Labels'] == 2]),2))
print('There are {} neighborhoods with 2 Italian Restaurants, they belong to cluster {}'.format(len(df_merged.loc[df_merged['Cluster Labels'] == 4]),4))
print('There are {} neighborhoods with 3 Italian Restaurants, they belong to cluster {}'.format(len(df_merged.loc[df_merged['Cluster Labels'] == 1]),1))
print('There are {} neighborhoods with 5 Italian Restaurants, they belong to cluster {}'.format(len(df_merged.loc[df_merged['Cluster Labels'] == 3]),3))

There are 55 neighborhoods with 0 Italian Restaurants, they belong to cluster 0
There are 28 neighborhoods with 1 Italian Restaurants, they belong to cluster 2
There are 6 neighborhoods with 2 Italian Restaurants, they belong to cluster 4
There are 6 neighborhoods with 3 Italian Restaurants, they belong to cluster 1
There are 1 neighborhoods with 5 Italian Restaurants, they belong to cluster 3


In [45]:
kclusters = 3
df_clustering = df_italian.drop(["Neighborhoods"], 1)
# Run k-means clustering algorithm
kmeans2 = KMeans(n_clusters=kclusters,random_state=0).fit(df_clustering)

In [46]:
# Creating a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
df_merged2 = df_italian.copy()
# Add the clustering labels
df_merged2["Cluster Labels"] = kmeans2.labels_
df_merged2.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
df_merged2

,Neighborhood,Italian Restaurant,Cluster Labels
0,Grünau,0,1
1,Adlershof,1,0
2,Alt-Hohenschönhausen,0,1
3,Alt-Treptow,2,0
4,Altglienicke,0,1
5,Baumschulenweg,1,0
6,Biesdorf,0,1
7,Blankenburg,0,1
8,Blankenfelde,0,1
9,Bohnsdorf,0,1


In [47]:
# Adding latitude and longitude values to the existing dataframe
df_merged2['Latitude'] = df['Latitude']
df_merged2['Longitude'] = df['Longitude']
# Sorting the results by Cluster Labels
df_merged2.sort_values(["Cluster Labels"], inplace=True)
pd.set_option('display.max_rows', None)
df_merged2

,Neighborhood,Italian Restaurant,Cluster Labels,Latitude,Longitude
88,Wedding,2,0,52.44329,13.50272
23,Friedrichshagen,1,0,52.54212,13.54389
93,Wilmersdorf,1,0,52.48439,13.47957
26,Gatow,1,0,52.52119,13.42414
29,Grunewald,1,0,52.54781,13.35473
31,Halensee,1,0,52.42168,13.43099
66,Pankow,1,0,52.53960,13.14282
65,Oberschöneweide,1,0,52.53487,13.20216
21,Friedenau,1,0,52.50302,13.58079
63,Niederschönhausen,1,0,52.45324,13.14345


In [48]:
# Creating the map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# Setting color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged2['Latitude'], df_merged2['Longitude'], df_merged2['Neighborhood'], df_merged2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat,lon],radius=5,popup=label,color=rainbow[cluster-1],fill=True,fill_color=rainbow[cluster-1],fill_opacity=0.7).add_to(map_clusters)
map_clusters

In [49]:
df_summary2=df_merged2.drop_duplicates(subset = ["Italian Restaurant"]).sort_values(['Italian Restaurant'],ascending=True)
df_summary2

,Neighborhood,Italian Restaurant,Cluster Labels,Latitude,Longitude
91,Wilhelmsruh,0,1,52.45549,13.51328
23,Friedrichshagen,1,0,52.54212,13.54389
88,Wedding,2,0,52.44329,13.50272
47,Lichtenberg,3,2,52.58887,13.36949
72,Rudow,5,2,52.45784,13.31808


In [53]:
print('There are {} neighborhoods with 0 Italian Restaurants, they belong to cluster {}'.format(len(df_merged2.loc[df_merged2['Cluster Labels'] == 1]),1))
print('There are {} neighborhoods with 1 or 2 Italian Restaurants, they belong to cluster {}'.format(len(df_merged2.loc[df_merged2['Cluster Labels'] == 0]),0))
print('There are {} neighborhoods with 3 or 5 Italian Restaurants, they belong to cluster {}'.format(len(df_merged2.loc[df_merged2['Cluster Labels'] == 2]),2))

There are 55 neighborhoods with 0 Italian Restaurants, they belong to cluster 1
There are 34 neighborhoods with 1 or 2 Italian Restaurants, they belong to cluster 0
There are 7 neighborhoods with 3 or 5 Italian Restaurants, they belong to cluster 2


In [42]:
df_merged3=df_merged2
for i in range(len(df_merged3)):
    if df_merged3.loc[i, 'Italian Restaurant']==1:
        df_merged3.loc[i,'Cluster Labels']=1
df_merged3

,Neighborhood,Italian Restaurant,Cluster Labels,Latitude,Longitude
88,Wedding,2,0,52.44329,13.50272
23,Friedrichshagen,1,1,52.54212,13.54389
93,Wilmersdorf,1,1,52.48439,13.47957
26,Gatow,1,1,52.52119,13.42414
29,Grunewald,1,1,52.54781,13.35473
31,Halensee,1,1,52.42168,13.43099
66,Pankow,1,1,52.53960,13.14282
65,Oberschöneweide,1,1,52.53487,13.20216
21,Friedenau,1,1,52.50302,13.58079
63,Niederschönhausen,1,1,52.45324,13.14345


In [39]:
df_summary3=df_merged3.drop_duplicates(subset = ["Italian Restaurant"]).sort_values(['Italian Restaurant'],ascending=True)
df_summary3

,Neighborhood,Italian Restaurant,Cluster Labels,Latitude,Longitude
91,Wilhelmsruh,0,1,52.45549,13.51328
23,Friedrichshagen,1,1,52.54212,13.54389
88,Wedding,2,0,52.44329,13.50272
47,Lichtenberg,3,2,52.58887,13.36949
72,Rudow,5,2,52.45784,13.31808


In [40]:
# Creating the map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# Setting color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_merged3['Latitude'], df_merged3['Longitude'], df_merged3['Neighborhood'], df_merged3['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat,lon],radius=5,popup=label,color=rainbow[cluster-1],fill=True,fill_color=rainbow[cluster-1],fill_opacity=0.7).add_to(map_clusters)
map_clusters

In [54]:
print('There are {} neighborhoods with 0 or 1 Italian Restaurants, they belong to cluster {}'.format(len(df_merged3.loc[df_merged3['Cluster Labels'] == 1]),1))
print('There are {} neighborhoods with 2 Italian Restaurants, they belong to cluster {}'.format(len(df_merged3.loc[df_merged3['Cluster Labels'] == 0]),0))
print('There are {} neighborhoods with 3 or 5 Italian Restaurants, they belong to cluster {}'.format(len(df_merged3.loc[df_merged3['Cluster Labels'] == 2]),2))

There are 83 neighborhoods with 0 or 1 Italian Restaurants, they belong to cluster 1
There are 6 neighborhoods with 2 Italian Restaurants, they belong to cluster 0
There are 7 neighborhoods with 3 or 5 Italian Restaurants, they belong to cluster 2
